# Using First-order LINE to embed Cora
In this tutorial, we will see how to use the First-order LINE
node embedding method to compute embedding of the Cora citation graph.
We will retrieve the graph, compute its report, and the get visualizations, both
single-plot, complete and animated.

We will show how to get visualizations both using the most immediate
approach, simply using the `GraphVisualizer` object with the embedding method
name, and the slightly longer approach creating the embedding model and
computing the embedding.

Of note, GRAPE provides multiple implementations of the First-order LINE 
model, using as backend the Ensmallen and TensorFlow libraries, and we will show how to use all of them.

## Installing the library
First of all, to install GRAPE just run as usual:

```bash
pip install grape
```

In [1]:
!pip install -qU grape

## Retrieving the graph
To retrieve the Cora graph from the LINQS laboratory graph repository
we run the following line of code:

In [2]:
from grape.datasets.linqs import Cora
complete_graph = Cora()

Do note that we support the retrieval of many other graphs, 83644 at the time of writing,
and you just need to import the graph you desire from the `datasets` submodule.

Remember that you can peruse the complete list of available graphs by running:

In [3]:
from grape.datasets import get_all_available_graphs_dataframe
all_available_graphs_dataframe = get_all_available_graphs_dataframe(verbose=False)
all_available_graphs_dataframe

repository          name  \
0      pheknowlatorkg  PheKnowLator   
1      pheknowlatorkg  PheKnowLator   
2      pheknowlatorkg  PheKnowLator   
3      pheknowlatorkg  PheKnowLator   
4      pheknowlatorkg  PheKnowLator   
...               ...           ...   
83639        wikidata      WikiData   
83640        wikidata      WikiData   
83641        wikidata      WikiData   
83642        wikidata      WikiData   
83643        wikidata      WikiData   

                                                 version  
0         v2.0.0-2020-5-10.instance-inverseRelations-owl  
1      v2.0.0-2020-5-10.instance-inverseRelations-owl...  
2            v2.0.0-2020-5-10.instance-relationsOnly-owl  
3        v2.0.0-2020-5-10.instance-relationsOnly-owlnets  
4         v2.0.0-2020-5-10.subclass-inverseRelations-owl  
...                                                  ...  
83639                      wikidata-20220713-truthy-BETA  
83640                     wikidata-20220715-lexemes-BETA  
83641                                         latest-all  
83642                                     latest-lexemes  
83643                                      latest-truthy  

[83644 rows x 3 columns]

Now, in the retrieved version of the Cora graph we find both nodes representing the
papers and nodes representing words present in the paper abstracts.

In most benchmarks, only the topology represented by the paper nodes is used and therefore
we run a filter that splits the graph into the topology of exclusively the paper nodes and
a dataframe containing the one-hot encoded words vectors.

In [4]:
from grape.datasets.linqs import get_words_data
graph, node_features = get_words_data(complete_graph)

## Graph report
Let's proceed to compute the graph report, which includes both the ordinary topological
information, such as the number of edges and number of nodes, number of connectes components
and number of singletons, plus many topological oddities.

We will see that Cora, even though it is commonly used as a node-label prediction 
benchmark in the literature, is filled with topological oddities:
holdouts that put in the test set nodes belonging to some of these oddities may
achieve significantly higher performance than less lucky holdouts, ad models that
exploit these oddities may appear as extremely well performing while not actually
achieving nothing but overfitting to these graph peculiarities.

In [5]:
graph

<div class="graph-report"><style>.graph-report li {margin: 0.5em 0 0.5em 0;}.graph-report .paragraph {text-align: justify;word-break: break-all;}.graph-report .small-columns {column-count: 4;column-gap: 2em;}.graph-report .medium-columns {column-count: 3;column-gap: 2em;}.graph-report .large-columns {column-count: 2;column-gap: 2em;}.graph-report .single-column {}@media only screen and (max-width: 600px) {.graph-report .small-columns {column-count: 1;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 600px) and (max-width: 800px) {.graph-report .small-columns {column-count: 2;}.graph-report .medium-columns {column-count: 1;}.graph-report .large-columns {column-count: 1;}}@media only screen and (min-width: 800px) and (max-width: 1024px) {.graph-report .small-columns {column-count: 3;}.graph-report .medium-columns {column-count: 2;}.graph-report .large-columns {column-count: 1;}}</style><h2>Cora</h2><p class="paragraph">The undirected graph Cora has 2.71K heterogeneous nodes and 5.28K edges. The graph contains 78 connected components, with the largest one containing 2.48K nodes and the smallest one containing 2 nodes. The RAM requirements for the nodes and edges data structures are 207.68KB and 16.34KB respectively.</p><h3 style="margin: 1em 0 0 0;">Degree centrality</h3><p class="paragraph">The minimum node degree is 1, the maximum node degree is 168, the mode degree is 2, the mean degree is 3.90 and the node degree median is 3.</p><p class="paragraph">The nodes with the highest degree centrality are 35 (degree 168 and node type Genetic_Algorithms), 6213 (degree 78 and node type Reinforcement_Learning), 1365 (degree 74 and node type Neural_Networks), 3229 (degree 65 and node type Neural_Networks) and 910 (degree 44 and node type Neural_Networks).</p><h3 style="margin: 1em 0 0 0;">Node types</h3><p class="paragraph">The graph has 7 node types, which are Neural_Networks (818 nodes, 30.21%), Probabilistic_Methods (426 nodes, 15.73%), Genetic_Algorithms (418 nodes, 15.44%), Theory (351 nodes, 12.96%), Case_Based (298 nodes, 11.00%), Reinforcement_Learning (217 nodes, 8.01%) and Rule_Learning (180 nodes, 6.65%). The RAM requirement for the node types data structure is 141.72KB.</p><h3 style="margin: 1em 0 0 0;">Topological Oddities</h3><p class="paragraph">A topological oddity is a set of nodes in the graph that <i>may be derived</i> by an error during the generation of the edge list of the graph and, depending on the task, could bias the results of topology-based models. In the following paragraph, we will describe the detected topological oddities.</p><h4 style="margin: 1em 0 0 0;">Node tuples</h4><p class="paragraph">A node tuple is a connected component composed of two nodes. We have detected 57 node tuples in the graph, involving a total of 114 nodes (4.21%) and 57 edges (0.54%). The detected node tuples are:</p><ul class="medium-columns"><li><p class="paragraph">Node tuple containing the nodes 1105622 (node type Neural_Networks) and 430574 (node type Neural_Networks).</p></li>
<li><p class="paragraph">Node tuple containing the nodes 116512 (node type Neural_Networks) and 1107808 (node type Neural_Networks).</p></li>
<li><p class="paragraph">Node tuple containing the nodes 1107728 (node type Neural_Networks) and 115188 (node type Neural_Networks).</p></li>
<li><p class="paragraph">Node tuple containing the nodes 1136040 (node type Neural_Networks) and 754594 (node type Neural_Networks).</p></li>
<li><p class="paragraph">Node tuple containing the nodes 73972 (node type Case_Based) and 50980 (node type Case_Based).</p></li>
<li><p class="paragraph">Node tuple containing the nodes 628458 (node type Neural_Networks) and 628459 (node type Neural_Networks).</p></li>
<li><p class="paragraph">Node tuple containing the nodes 180301 (node type Probabilistic_Methods) and 1110628 (node type Probabilistic_Methods).</p></li>
<li><p class="paragraph">Node tuple c

## Graph visualizations
In GRAPE we support the visualization of a wide variety of graph properties,
which may either visualized through PCA, TSNE or UMAP decompositions, as a single plot
per property or as a composite plot. Furthermore, single plots may animated through
a high-dimensional rotation that allows for a more comprensive view of the embedding
decompositions.

### Computing embedding and plotting the node types
We start by showing how to use the `GraphVisualizer` object to compute in a few lines of code
a provided node embedding method, and to use the TSNE decomposition to display the node
types both in 2D and 4D. The 4D animation is achieved by executing a 4D decomposition,
rotating the decomposition in 4D and then plot the first 3 dimensions in a 3-dimensional plot.

#### Visualization using the Ensmallen library

In [6]:
from grape import GraphVisualizer
visualizer = GraphVisualizer(graph)

# You can either provide the model name
visualizer.fit_nodes("First-order LINE", library_name="Ensmallen")

# Or provide a precomputed embedding (here commented)
#
# visualizer.fit_nodes(numpy_array_with_embedding)
# visualizer.fit_nodes(pandas_dataframe_with_embedding)
#
# or alternatively provide the model to be used:
#
# from grape.embedders import FirstOrderLINEEnsmallen
# visualizer.fit_nodes(FirstOrderLINEEnsmallen())
#
# And now we can visualize the node types:
visualizer.plot_node_types()

For the 4D animation we will be rendering a webm,
but most video formats are supported. Analogously to the bidimensional
image, we run:

In [7]:
visualizer = GraphVisualizer(
    graph,
    n_components=4,
    rotate=True
)
visualizer.fit_nodes("First-order LINE", library_name="Ensmallen")
visualizer.plot_node_types()

OpenCV: FFMPEG: tag 0x30387076/'vp80' is not supported with codec id 139 and format 'webm / WebM'


To obtain instead the complete plot with all of the properties visualized,
you may run the following one-liner:

In [8]:
GraphVisualizer(graph).fit_and_plot_all("First-order LINE", library_name="Ensmallen")
        

Euclidean distance 0.09697365 1.8953899
Cosine similarity -0.7954107 0.9952985
Adamic-Adar 0.0 11.12674
Jaccard Coefficient 0.0 0.6666667
Preferential Attachment 0.0 0.44045636
Resource Allocation Index 0.0 3.7750003


#### Visualization using the TensorFlow library

In [9]:
from grape import GraphVisualizer
visualizer = GraphVisualizer(graph)

# You can either provide the model name
visualizer.fit_nodes("First-order LINE", library_name="TensorFlow")

# Or provide a precomputed embedding (here commented)
#
# visualizer.fit_nodes(numpy_array_with_embedding)
# visualizer.fit_nodes(pandas_dataframe_with_embedding)
#
# or alternatively provide the model to be used:
#
# from grape.embedders import FirstOrderLINETensorFlow
# visualizer.fit_nodes(FirstOrderLINETensorFlow())
#
# And now we can visualize the node types:
visualizer.plot_node_types()

2022-08-03 12:44:45.117226: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 12:44:45.159388: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 12:44:45.159622: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 12:44:45.175026: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

For the 4D animation we will be rendering a webm,
but most video formats are supported. Analogously to the bidimensional
image, we run:

In [10]:
visualizer = GraphVisualizer(
    graph,
    n_components=4,
    rotate=True
)
visualizer.fit_nodes("First-order LINE", library_name="TensorFlow")
visualizer.plot_node_types()

OpenCV: FFMPEG: tag 0x30387076/'vp80' is not supported with codec id 139 and format 'webm / WebM'


To obtain instead the complete plot with all of the properties visualized,
you may run the following one-liner:

In [11]:
GraphVisualizer(graph).fit_and_plot_all("First-order LINE", library_name="TensorFlow")
        

Euclidean distance 0.21506602 4.776469
Cosine similarity -0.41463104 0.98203385
Adamic-Adar 0.0 11.12674
Jaccard Coefficient 0.0 0.6666667
Preferential Attachment 0.0 0.44045636
Resource Allocation Index 0.0 3.7750003


## Embedding the graph
Compute the embedding of the Cora graph using the First-order LINE model.

### Embedding using the Ensmallen library implementation

In [12]:
from grape.embedders import FirstOrderLINETensorFlow
embedding = FirstOrderLINETensorFlow().fit_transform(graph)

### Embedding using the TensorFlow library implementation

In [13]:
from grape.embedders import FirstOrderLINETensorFlow
embedding = FirstOrderLINETensorFlow().fit_transform(graph)